## first step is to make movies and lighttime out of single images 

In [ ]:
# enter the folder name 
Mainfolder = 'C:\\videos\\ar30motor\\2021_07_09';
#C:\videos\ar32motor\2021_06_11#Mainfolder = 'C:\\videos\\ar32motor\\2021_04_25';

Mainfolder = 'D:\\Sidevideos\\ar30motor\\2021_07_09'
Mainfolder = 'D:\\Sidevideos\\ar34secondday\\2021_07_15'
Mainfolder = 'D:\\Sidevideos\\AR34MOTOR\\2022_01_27'
#Mainfolder =  'D:\\Sidevideos\\ar34Lightduration\\2021_08_03'
import main



In [ ]:
#main.make_movie_only(Mainfolder)
main.makemovieandLightfile(Mainfolder)
main.save_trial_n(Mainfolder)

In [ ]:
import deeplabcut
## https://github.com/AlexEMG/DeepLabCut

In [ ]:
path_config_file = 'D:\\Sidevideos\\DLC\\Sideview4-Arash-2021-07-18\\config.yaml';
#Mainfolder = 'D:\\videos\\ar32\\2020_12_18';
path_config_file ='C:\\Users\\dklab\\Dropbox\\Notebook\\DLC\\Sideview_Aug-Arash-2021-08-04\\config.yaml';
#Mainfolder = 'D:\\videos\\ar32\\2020_12_18';
import os
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
print(text_files[1])
#text_files='C:\\videos\\ar30motor\\2021_04_25\\9.avi'
temp1=deeplabcut.analyze_videos(path_config_file,text_files,shuffle=1, save_as_csv=True )
temp=deeplabcut.filterpredictions(path_config_file,text_files)

In [1]:
Mainfolder = 'D:\\Sidevideos\\ar34motor\\2021_12_16'
Mainfolder = 'D:\\Sidevideos\\AR34MOTOR\\2022_01_27'
import image_util
#Mainfolder = 'F:\\ar34motor\\sideright\\2021_12_16'
trial=26
Tag ='Sideview'
from PIL import Image ,ImageEnhance
from scipy import ndimage
import numpy as np
import cv2 # this is important and is reading and writing images and video
import image_util
import os
from pathlib import Path
import datetime
import math
import image_util
# this function read dlc 
#def readDLCfiles(Mainfolder,Tag,trial):
#11185DLC_resnet50_Topview3435Jul28shuffle1_110000_filtered
import csv
import os
import pandas as pd
from scipy.spatial.distance import pdist
import math
import matplotlib.pyplot as plt

Xfiles = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('filtered.csv') and Tag in f] 
# csv file name
#for num in range(0,len(Xfiles)): 
for num in range(0,1): 
    filename = Xfiles[num] # select the trial
    df = pd.read_csv(filename, header=2 ,usecols=['x','y','likelihood','x.1','y.1','likelihood.1','x.2','y.2','likelihood.2'])
    df.columns=  ['Nosex','Nosey','Noselikelihood','Snoutx1','Snouty1','Snoutlikelihood','Eyex','Eyey','Eyelikelihood']

    # smooth the xy coordinates by simple avaraging 
    smoothingwin = 5;
    movivename = filename;
    x3 = image_util.smooth_data_convolve_my_average(df.Snoutx1, smoothingwin);
    y3 = image_util.smooth_data_convolve_my_average(df.Snouty1, smoothingwin);
    smoothingwin = 5;
    x1 = image_util.smooth_data_convolve_my_average(df.Nosex, smoothingwin);
    y1 = image_util.smooth_data_convolve_my_average(df.Nosey, smoothingwin);
    x2 = image_util.smooth_data_convolve_my_average(df.Snoutx1, smoothingwin);
    y2 = image_util.smooth_data_convolve_my_average(df.Snouty1, smoothingwin);
    Extention = '.avi'

    Angles = [math.atan2(-(y1[i]-y2[i]),-(x1[i]-x2[i]))  for i in range(len(df.Snoutlikelihood))] # define the angle of the head
    Distance = [math.sqrt((x2[i] - x1[i])**2 + (y2[i] - y1[i])**2)  for i in range(len(df.Snoutlikelihood))]# define the distance between beads  
    Angle=pd.Series(Angles)
    #Angles[Angles<-2]= Angles[Angles<-2]+2*math.pi;

    text = os.path.basename(movivename);
    video_name = (os.path.join(os.path.dirname(movivename),text.split('DLC')[0]+Extention));
    video_nameEYE = (os.path.join(os.path.dirname(movivename),text.split('DLC')[0]+"EYE.avi"));

    cap = cv2.VideoCapture(video_name)# reading the videofile
    frame_width  = np.uint8(cap.get(3)) # float
    frame_height = np.uint8(cap.get(4)) # float
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    i=0
    factor=2;
    video = cv2.VideoWriter(video_nameEYE, 0, 40, (200,200))

    faceshift=60;
    while(cap.isOpened()):
     ret, frame = cap.read() 
     if ret == True:
        i+=1
        height, width, layers = frame.shape
        #rotation angle in degree
        if 1==1:
            #rotated = ndimage.rotate(frame, math.degrees(Angle[i-1])-90+180)
            color_coverted = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            sizetotal = 200;
            midpoint= 100;  
            img = Image.fromarray(color_coverted, 'RGB')
            Newrotated=np.uint8(image_util.add_margin(img, 100, 100, 100, 100, (0,0,0)))
            y= int(df.Eyey[i-1]+100-midpoint);
            x= int(df.Eyex[i-1]+100-midpoint);
            h = sizetotal
            w = int(sizetotal)
            crop_img = Newrotated[y:y+h, x:x+w]
            #frame2 = cv2.flip(crop_img, 1)
            #frame2 = image_util.mask(crop_img,60)
            #frame2 = Image.fromarray(frame2)
            #frame2 = frame2.convert("RGB")
            #enhancer = ImageEnhance.Contrast(frame2)
            #im_output = enhancer.enhance(factor)
            video.write(np.array(crop_img))

        #Mirror92R

     else:
        break
    video.release()

In [ ]:

Mainfolder = 'F:\\ar34motor\\sideright\\2021_12_16'
trial=223
Tag ='Sideview'
Xfiles = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('filtered.csv') and Tag in f] 
# csv file name
filename = Xfiles[trial] # select the trial
print(filename)

In [ ]:
print(video_nameEYE)
sizetotal = 200;
midpoint= 100;  
cap = cv2.VideoCapture(video_name)# reading the videofile

# Implementation of matplotlib function
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
i=0
while(cap.isOpened()):
 ret, frame = cap.read() 
 if ret == True:
    i+=1
    height, width, layers = frame.shape
    #rotation angle in degree
    if 1==1:
        #rotated = ndimage.rotate(frame, math.degrees(Angle[i-1])-90+180)
        color_coverted = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        sizetotal = 200;
        midpoint= 100;  
        img = Image.fromarray(color_coverted, 'RGB')
        Newrotated=np.uint8(add_margin(img, 100, 100, 100, 100, (0,0,0)))
        y= int(df.Eyey[i-1]+100-midpoint);
        x= int(df.Eyex[i-1]+100-midpoint);
        h = sizetotal
        w = int(sizetotal)
        crop_img = Newrotated[y:y+h, x:x+w]
        c = plt.imshow(crop_img)
 
        plt.show()

    #Mirror92R

 else:
    break
    
    


In [ ]:
crop_img = Newrotated[y:y+h, x:x+w]
c = plt.imshow(color_coverted)
plt.plot(df.Eyex[i-1],df.Eyey[i-1] ,'rp', markersize=14)



plt.show()



Newrotated=np.uint8(add_margin(img, 100, 100, 100, 100, (0,0,0)))
y= int(df.Eyey[i-1]);
x= int(df.Eyex[i-1]);
c = plt.imshow(Newrotated)
plt.plot(x,y ,'rp', markersize=14)
plt.show()

In [ ]:
print(x)

In [ ]:

def smooth_data_convolve_my_average(arr, span):
    import numpy as np
    re = np.convolve(arr, np.ones(span * 2 + 1) / (span * 2 + 1), mode="same")
        # The "my_average" part: shrinks the averaging window on the side that 
    # reaches beyond the data, keeps the other side the same size as given 
    # by "span"
    re[0] = np.average(arr[:span])
    for i in range(1, span + 1):
        re[i] = np.average(arr[:i + span])
        re[-i] = np.average(arr[-i - span:])
    return re
def crop_rotated(rotated,frame,Angle,i,df):
    from PIL import Image
    import numpy as np
    import math
    img = Image.fromarray(rotated, 'RGB')
    Newrotated=np.uint8(add_margin(img, 400, 400, 400, 400, (0,0,0)))
    Alpha = math.degrees(Angle[i])-90+180;
    Alpharad = math.radians(math.degrees(Angle[i])-90+180);
    utP = [df.Nosex[i] ,df.Nosey[i]]
    P = [df.Nosey[i] ,df.Nosex[i]]
    #P = [thiscenter[2],thiscenter[1]]   #coordinates of point
    c, s = np.cos(Alpharad),np.sin(Alpharad)
    RotMatrix = np.array(((c, -s), (s, c)))
    ImCenterA = np.array(frame.shape[0:2])/2       # Center of the main image
    ImCenterB = np.array(Newrotated.shape[0:2])/2  # Center of the transformed image
    RotatedP =RotMatrix.dot(P-ImCenterA)+ImCenterB;
    midpoint= 350;
    sizetotal = 700;
    ratsiosize = 1.1;
    y= int(RotatedP[0]-midpoint);
    x= int(RotatedP[1]-midpoint*ratsiosize);
    h = sizetotal
    w = int(sizetotal*ratsiosize)
    crop_img = Newrotated[y:y+h, x:x+w]
    return crop_img
def add_margin(pil_img, top, right, bottom, left, color):
    from PIL import Image
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

## this step is to load deeplabcut and run already trained algorithm on new movies to track the face

In [ ]:
import deeplabcut
## https://github.com/AlexEMG/DeepLabCut

In [ ]:
path_config_file = 'D:\\Sidevideos\\DLC\\Sideview4-Arash-2021-07-18\\config.yaml';
#Mainfolder = 'D:\\videos\\ar32\\2020_12_18';
path_config_file ='D:\\Sidevideos\\DLC\\Sideview_Aug-Arash-2021-08-04\\config.yaml';
#Mainfolder = 'D:\\videos\\ar32\\2020_12_18';
import os
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
print(text_files[1])
#text_files='C:\\videos\\ar30motor\\2021_04_25\\9.avi'
temp1=deeplabcut.analyze_videos(path_config_file,text_files,shuffle=1, save_as_csv=True )
temp=deeplabcut.filterpredictions(path_config_file,text_files)

In [ ]:
print(text_files)


# this step should take the filtered DLC output (Nose and Snout)
and create the L and R movies
then using another function to convert L and R (R is left face and L is right face) files into Mask and Mirror files( Mask only adds mask to the Right side of the face and Mirror. 


# copy tracked files into the server (with mp4 movies)

In [ ]:
# the code to run matlab here
#Mainfolder = 'C:\\videos\\ar344thday\\2021_07_26'
# copy tracked files into the server (with mp4 movies)
Mainfolder = 'D:\\Sidevideos\\ar32motor\\2021_08_02'

#Mainfolder = 'D:\\Sidevideos\\ar344thday\\2021_07_26'
#destination = ''
destination='//dk-server.dk.ucsd.edu/afassihizakeri/SC_Movies/ar32motor/2021_08_02/Side'
import shutil
shutil.copytree( Mainfolder,destination, ignore=shutil.ignore_patterns('*.avi'),copy_function = shutil.copy)
#copytree(Mainfolder, destination, ignore=ignore_patterns('*.avi'))




this step makes Mirror Mask files

In [ ]:
import Runmask_mirror

#Mainfolder = 'D:\\videos\\ar32\\2020_11_24\\'
#Mainfolder = 'D:\\videos\\ar32\\2020_12_18\\'

Runmask_mirror.RunMask(Mainfolder)

# now apply DLC already trained data on whiskers 

In [ ]:

#Mainfolder = 'D:\\videos\\ar32\\2020_11_24'
import image_util
path_config_file = 'D:\\videos\\ar30\\2020_11_04\\ar30_11_05_20whiskers-Arash-2020-11-08\\config.yaml'


[XfilesL,XfilesR]=image_util.get_mask_mirror_names(Mainfolder) 


scorernameL=deeplabcut.analyze_videos(path_config_file,XfilesL,shuffle=1, save_as_csv=True)
print(scorernameL)
deeplabcut.filterpredictions(path_config_file,XfilesL,shuffle=1)
scorernameR=deeplabcut.analyze_videos(path_config_file,XfilesR,shuffle=1, save_as_csv=True)
print(scorernameR)
deeplabcut.filterpredictions(path_config_file,XfilesR,shuffle=1)

# make  movies and plot rajectories from the dlc data

In [ ]:
path_config_file = 'D:\\videos\\ar32\\ar32_10_11_24-Arash-2020-11-25\\config.yaml';
path_config_file = 'D:\\Sidevideos\\DLC\\Sideview4-Arash-2021-07-18\\config.yaml';

Mainfolder = 'D:\\videos\\ar30\\2021_01_09';
Mainfolder = 'D:\\Sidevideos\\ar343rdday\\2021_07_16'
import os
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('Tracked.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
videofile = os.path.join(Mainfolder,'119.avi')
print(text_files)
del text_files[0]
deeplabcut.plot_trajectories(path_config_file,text_files,filtered = True)
deeplabcut.create_labeled_video(path_config_file,text_files,save_frames=False)

# copy tracked data

In [ ]:
import os  # this is to get folder information and creat files
import cv2  # this is important and is reading and writing images and video
import shutil
destination='//dk-server.dk.ucsd.edu/afassihizakeri/SC_Movies/ar34training/4thdayside'

All_Folders=[ name for name in os.listdir(Mainfolder) if not os.path.isdir(os.path.join(Mainfolder, name)) ]
#print(All_Folders)
for num in range(0,len(All_Folders)): 
    thisdestination = os.path.join(destination+'/', All_Folders[num])
    thissource = os.path.join(Mainfolder+os.sep, All_Folders[num])
    shutil.copy( thissource,destination)
    print(thisdestination)


# this part should take scorername L and R as an input to matlab 
import image_util

In [ ]:
print(scorernameL)
do matlab thing

In [ ]:
Mainfolder = 'D:\\videos\\ar30\\2021_01_09\\'

main.save_trial_n(Mainfolder)

the last step is to check everything